In [1]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from loguru import logger

/opt/homebrew/Caskroom/miniconda/base/envs/dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size, shuffle=True, drop_last=True)


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
model = NeuralNetwork()
model = model.to(device)

In [7]:
loss_func = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), 2e-4)

In [8]:
EPOCH = 5
for i in range(EPOCH):
    logger.info(f"Epoch {i+1}")
    for idx, (x,y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        loss = loss_func(y_pred, y)
        loss.backward()
        opt.step()
        opt.zero_grad()
        if idx % 100 == 0:
            logger.info(f"loss{loss.item()}")

2024-02-05 00:07:13.551 | INFO     | __main__:<module>:3 - Epoch 1
2024-02-05 00:07:16.029 | INFO     | __main__:<module>:13 - loss2.306499481201172
2024-02-05 00:07:16.739 | INFO     | __main__:<module>:13 - loss0.9275075197219849
2024-02-05 00:07:17.426 | INFO     | __main__:<module>:13 - loss0.6783807277679443
2024-02-05 00:07:18.113 | INFO     | __main__:<module>:13 - loss0.515682578086853
2024-02-05 00:07:18.944 | INFO     | __main__:<module>:13 - loss0.4708956480026245
2024-02-05 00:07:19.744 | INFO     | __main__:<module>:13 - loss0.49175596237182617
2024-02-05 00:07:20.466 | INFO     | __main__:<module>:13 - loss0.6429104804992676
2024-02-05 00:07:21.157 | INFO     | __main__:<module>:13 - loss0.6625133752822876
2024-02-05 00:07:21.871 | INFO     | __main__:<module>:13 - loss0.43106377124786377
2024-02-05 00:07:22.654 | INFO     | __main__:<module>:13 - loss0.42065590620040894
2024-02-05 00:07:22.943 | INFO     | __main__:<module>:3 - Epoch 2
2024-02-05 00:07:22.952 | INFO     

In [11]:
test_loss = 0
correct = 0
with torch.no_grad():
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        test_loss += loss_func(pred, y).item()
        correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()
test_loss = test_loss / len(test_dataloader)
correct = correct / len(test_dataloader.dataset)
logger.info(f"Avg loss is {test_loss}, correct is {correct}")

2024-02-05 11:52:44.921 | INFO     | __main__:<module>:11 - Avg loss is 0.3726440172355909, correct is 0.8652
